In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch

2024-04-22 00:02:31.188374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 00:02:31.775916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [4]:
mirrored_strategy = tf.distribute.MirroredStrategy()

2024-04-22 00:02:41.174404: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45741 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-04-22 00:02:41.175696: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 45361 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:23:00.0, compute capability: 8.6
2024-04-22 00:02:41.176755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 45361 MB memory:  -> device: 2, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6
2024-04-22 00:02:41.177618: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 45361 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:61:00.0, 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [22]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [23]:
#실제 모델에서 제공하는 generate (추론 과정))
sent = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
with mirrored_strategy.scope():
    output = model.generate(input_ids,
                        max_length=100,
                        use_cache=False)
    output_ids = output.numpy().tolist()[0]
    print(tokenizer.decode(output_ids))

#=====여기부터 워터마킹======
#아이들의 정서적인 발달에 동화책은 큰 도움이 될 수 있다.
#또한 동화책 읽기를 통해 아이들의 정서적인 발달에 도움을 줄 수 있다.
#동화책 읽기를 통해 아이들은 자신의 생각을 자유롭게 표현할 수 있고, 자신의 생각을 자유롭게 표현할 수 있다.

아이들의 정서적인 발달에 동화책은 큰 도움이 될 수 있다.
또한 동화책 읽기를 통해 아이들의 정서적인 발달에 도움을 줄 수 있다.
동화책 읽기를 통해 아이들은 자신의 생각을 자유롭게 표현할 수 있고, 자신의 생각을 자유롭게 표현할 수 있다.
또한 동화책을 통해 자신의 생각을 자유롭게 표현할 수 있다.
동화책 읽기를 통해 아이들은 자신의 생각을 자유롭게 표현할 수 있고, 자신의 생각을 자유롭게 표현할 수 있다.
동화책 읽기를 통해 아이들은 자신의 생각을 자유롭게 표현할 수 있고, 자신의 생각을 자유롭게 표현할 수 있다.
동화책 읽기를 통해 아이들은 자신의 생각을 자유롭게


In [7]:
sent = '아이들의 정서적인 발달에 동화책은'

In [8]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[42738 17381  9090 24004 19383 16095]], shape=(1, 6), dtype=int32)


In [9]:
#output = model.generate(input_ids,
#                        max_length=128,
#                        repetition_penalty=2.0,
#                        use_cache=True)

output = model(input_ids)
Logits = output.logits
Logits

<tf.Tensor: shape=(1, 6, 51200), dtype=float32, numpy=
array([[[-6.3650374 , -5.9244194 , -5.5607867 , ...,  0.78603697,
         -5.022273  , -2.9620056 ],
        [-7.120081  , -7.1628304 , -6.602664  , ..., -2.2832334 ,
         -5.9630513 , -4.78648   ],
        [-7.444871  , -6.6141295 , -4.999629  , ...,  0.51350355,
         -2.3515596 , -4.5314565 ],
        [-5.8011975 , -6.3517947 , -5.1028595 , ...,  2.6300447 ,
         -0.4167254 , -1.6408297 ],
        [-6.1230216 , -6.6718645 , -5.732671  , ...,  1.2544551 ,
         -0.70743334, -2.309436  ],
        [-5.313277  , -5.550266  , -5.276083  , ...,  5.036843  ,
          0.1513547 , -2.1300857 ]]], dtype=float32)>

In [10]:
Logits.shape

TensorShape([1, 6, 51200])

In [12]:
#알고리즘 설정하지 않은 것, Greedy search
input_text = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(input_text, return_tensors='tf')

with mirrored_strategy.scope(): # Logit 계속 받아오는거랑 generate 랑 같은 지 보기
    while input_ids.shape[-1] < 100:
        print("================================")
        last_index = input_ids.shape[-1]
        print(input_ids.shape[-1])
        Logits = model(input_ids).logits
        print(Logits)
        
        add_token_id = tf.argmax(Logits[0][last_index-1][:])
        print(add_token_id)
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)

        next_sequence = tokenizer.decode(input_ids.numpy()[0,:])
        print(next_sequence)
    
        


6
tf.Tensor(
[[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  [-5.8011975  -6.3517947  -5.1028595  ...  2.6300447  -0.4167254
   -1.6408297 ]
  [-6.1230216  -6.6718645  -5.732671   ...  1.2544551  -0.70743334
   -2.309436  ]
  [-5.313277   -5.550266   -5.276083   ...  5.036843    0.1513547
   -2.1300857 ]]], shape=(1, 6, 51200), dtype=float32)
tf.Tensor(9458, shape=(), dtype=int64)
아이들의 정서적인 발달에 동화책은 큰
7


tf.Tensor(
[[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  ...
  [-6.1230216  -6.6718645  -5.732671   ...  1.2544551  -0.70743334
   -2.309436  ]
  [-5.313277   -5.550266   -5.276083   ...  5.036843    0.1513547
   -2.1300857 ]
  [-6.5101585  -6.136811   -6.118112   ...  2.4861598  -3.2500486
   -8.959919  ]]], shape=(1, 7, 51200), dtype=float32)
tf.Tensor(13229, shape=(), dtype=int64)
아이들의 정서적인 발달에 동화책은 큰 도움이
8
tf.Tensor(
[[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  ...
  [-5.313277   -5.550266   -5.276083   ...  5.036843    0.1513547
   -2.1300857 ]
  [-6.5101585  -6.136811   -6.118112   ..

In [14]:
input_text = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(input_text, return_tensors='tf')
#term = 3

last_index = input_ids.shape[-1]

with mirrored_strategy.scope():
    while input_ids.shape[-1] < 100:
        print("================================")
        print(input_ids.shape[-1])
        #set 나눠서 green set 로짓을 촉진하기
        gamma = 1.0
        delta = 0.0
        Logits = model(input_ids).logits
        print(f"original Logits : {Logits}")
        
        #seed 생성
        seed = input_ids[-1] % Logits.shape[-1]
        print(f"seed : {seed[-1]}")
        np.random.seed(seed[-1])

        #random green list 설정
        indices = np.arange(Logits.shape[-1]) #0~51999
        green_list_size = int(Logits.shape[-1] * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
        print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

        #--여기부터 probability 구하기
        # green logit과 red logit의 값을 담을 리스트 생성
        green_logits = tf.zeros(green_list_size)
        red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

        #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
        Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
        Logits_num = Logits.numpy()
        for i in green_list :
            Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
        Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
        print(f"updated Logits : {Logits}")
        
        # softmax 계산을 위한 분모
        denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
        print(f"denominator : {denominator}") #25600개의 green list

        # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산)
        softmax_tensor = Logits.numpy()
        softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
        softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
        print(softmax_tensor.shape)
        print(f"Probabilities : {softmax_tensor}")

        # 데이터 타입 확인 및 조정
        add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
        print(f"updated Logits Max value : {add_token_id}")


        # 텐서 연결
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)
        next_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(f"next token : {next_token}")
        
        last_index = input_ids.shape[-1]
        #term = 3

6
original Logits : [[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  [-5.8011975  -6.3517947  -5.1028595  ...  2.6300447  -0.4167254
   -1.6408297 ]
  [-6.1230216  -6.6718645  -5.732671   ...  1.2544551  -0.70743334
   -2.309436  ]
  [-5.313277   -5.550266   -5.276083   ...  5.036843    0.1513547
   -2.1300857 ]]]
seed : 16095
green list : [33079 19547 38315 ... 42505 30137 32538] (51200,)
updated Logits : [[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  [-5.8011975  -6.3517947  -5.1028595  ...  2.6300447  -0.4167254
   -1.6408297 ]
  [-6.1230216  -6.6718645  -5.732671   ...  1.2544551  -0.7074333

In [ ]:
#top5 = tf.math.top_k(output.logits[0, -1], k=5)